## Configuration of cairio.org

Right now the central cairio server is hosted on https://pairio.org:20443

This is a linode server

Inside a tmux session called cairio:

```
cd ~/src/spikeforest2/mountaintools/cairioserver
```

See .env file containing:

```
PORT=20443
CAIRIO_ADMIN_TOKEN=******
```

To run the server:

```
npm install .
node cairioserver.js
```

There are instructions in the home directory for setting up firewall and renewing the certs

In [1]:
%load_ext autoreload
%autoreload 2

# import cairio client and other packages
from cairio import client as ca
from cairio import CairioClient
from getpass import getpass
import os
from getpass import getpass
import json
from simplecrypt import encrypt, decrypt
import base64
from dotenv import load_dotenv

In [2]:
# Load secret stuff from the cairio.env file (only on the admin's computer)

load_dotenv(dotenv_path='cairio.env',verbose=True)
CAIRIO_ADMIN_TOKEN=os.environ['CAIRIO_ADMIN_TOKEN']

In [3]:
# Let's set up the admin collection where we will store the user login information
# It will have the same token as the admin token

ca.addRemoteCollection(collection='admin', token=CAIRIO_ADMIN_TOKEN, admin_token=CAIRIO_ADMIN_TOKEN)

True

In [4]:
# Now let's add some users.

def set_user_config(user,config):
    key=dict(
        name='user_config',
        user=user
    )
    val=base64.b64encode(encrypt(getpass('Enter an encryption password for {}: '.format(user)),json.dumps(config))).decode()
    cc=CairioClient()
    cc.setRemoteConfig(collection='admin',token=CAIRIO_ADMIN_TOKEN)
    cc.setValue(
        key=key,
        value=val
    )

# user=magland
config=dict(
    kbucket_shares=[
        dict(hub='kbucket.flatironinstitute.org',name='sf_raw',node_id='15734439d8cf'),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest1',node_id='7317cea8265b',upload_token=os.environ['SPIKEFOREST1_UPLOAD_TOKEN']),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest2',node_id='69432e9201d0',upload_token=os.environ['SPIKEFOREST2_UPLOAD_TOKEN'])
    ],
    cairio_collections=[
        dict(database='default',name='spikeforest',token=os.environ['SPIKEFOREST_TOKEN'])
    ]
)
set_user_config('magland',config)
set_user_config('jjun',config)

Enter an encryption password for magland:  ·····


CAIRIO CONFIG: remote database admin (readwrite); local sha-1 cache


Enter an encryption password for jjun:  ·····


CAIRIO CONFIG: remote database admin (readwrite); local sha-1 cache


In [5]:
# Now this is how we would log in as magland and configure the cairio client:

ca.login(user='magland', ask_password=True)
ca.configRemoteReadWrite(collection='spikeforest',share_id='69432e9201d0')

Cairio password for magland:  ·····


Logging in as magland...
Logged in as magland
CAIRIO CONFIG: remote database spikeforest (readwrite); remote kb-share 69432e9201d0 (readwrite)


In [6]:
# Or we can configure for readonly

ca.configRemoteReadonly(collection='spikeforest',share_id='69432e9201d0')

CAIRIO CONFIG: remote database spikeforest (readonly); remote kb-share 69432e9201d0 (readonly)


In [6]:
# Let's set the token for the remote spikeforest collection

collection='spikeforest'
ca.addRemoteCollection(collection=collection, token=os.environ['SPIKEFOREST_TOKEN'], admin_token=CAIRIO_ADMIN_TOKEN)

True

In [8]:
###### Let's do a test
ca.login(user='magland',ask_password=True)
ca.configRemoteReadWrite(collection='spikeforest',share_id='69432e9201d0')

ca.saveObject(key='test1',object=dict(test=1))
print(ca.loadObject(key='test1'))
ca.saveObject(key='test1',object=dict(test=2))
print(ca.loadObject(key='test1'))

Cairio password for magland:  ·····


Logging in as magland...
Logged in as magland
CAIRIO CONFIG: remote database spikeforest (readwrite); remote kb-share 69432e9201d0 (readwrite)
Testing whether share 69432e9201d0 is directly accessible...
Share 69432e9201d0 is directly accessible.
{'test': 1}
{'test': 2}
